# Q1

In [ ]:
import numpy as np
from astropy.timeseries import periodograms

In [31]:
def solve_kepler(e, M):
    """
    Solve Kepler's Equation M = E - e sinE
    """ 
    dEs = []
    E = M + 0.85 * e * np.sign(np.sin(M))
    dE = 1e3
    while abs(E - e * np.sin(E) - M) > 1e-10:
        dE = ((E - e * np.sin(E) - M)) / (1 - e * np.cos(E))
        E -= dE
    return E
        
    

Verify over 0 < e < 1 and 0 < M < 2$\pi$ that M = E - e sin(E)

In [39]:
e = np.random.random(size=10000)
M = 2 * np.pi * np.random.random(size=10000)

for e, M in zip(e, M):
    E = solve_kepler(e, M)
    assert(M - (E - e * np.sin(E)) < 1e-10)

# Q2

In [32]:
def true_anomaly(e, M):
    E = solve_kepler(e, M)
    return 2 * np.arctan(np.sqrt((1 + e) / (1 - e)) * np.tan(E / 2))

In [ ]:
def RV_solver(K_star, P, t, tp, e, omega, gamma):
    M = 2 * np.pi / P * (t - tp)
    f = true_anomaly(e, M)
    return K_star * (np.cos(omega) * np.cos(f) - np.sin(omega) * np.sin(f) + e * np.cos(e)) + gamma


# Q3